#  Syft Duet - Bob

In [23]:
# the length of iris dataset is 150
# how many samples would you let Bob hold?
data_owner = "Bob"
data_size = 80

In [24]:
#check data
if data_size <= 0:
    data_size=1
if data_size > 150:
    data_size=150

## PART 1: Launch a Duet Server and Connect

As a Data Owner, you want to allow someone else to perform data science on data that you own and likely want to protect.

In order to do this, we must load our data into a locally running server within this notebook. We call this server a "Duet".

To begin, you must launch Duet and help your Duet "partner" (a Data Scientist) connect to this server.

You do this by running the code below and sending the code snippet containing your unique Server ID to your partner and following the instructions it gives!

--from openmind course

In [25]:
#launch the server
import syft as sy
duet = sy.launch_duet(loopback=True)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.join_duet(loopback=True)

♫♫♫ > Connecting...


/home/fjunyuan/.local/lib/python3.8/site-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/home/fjunyuan/.local/lib/python3.8/site-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore


If you are in Jupyter Notebook (not Colab) the ☝🏾DUET LIVE STATUS above will animate 

# Part 2: Upload data to Duet Server

Let's say the data owner has a dataset of Iris flowers. He will upload the data to the duet server for other data scientists to use.

In [26]:
from sklearn import datasets
import torch as th

In [27]:
iris = datasets.load_iris()
X, y = iris.data, iris.target

In [28]:
print(type(X))

<class 'numpy.ndarray'>


In [29]:
#adjust size
if data_owner == "Alice":
    X = X[0:data_size]#Bob X  = X[(150-data_size):]
    y = y[0:data_size]#Bob y  = y[(150-data_size):]
if data_owner == "Bob":
    X  = X[(150-data_size):]
    y  = y[(150-data_size):]

For doing machine learning using torch, we need the data to be converted to FloatTensors. Here, the data owner is explicitly doing the conversion before uploading the data. If he doesn't do that, it has to be converted in the data scientist's end as you've seen in the previous exercise.

In [30]:
X = th.FloatTensor(X)
y = th.FloatTensor(y)

In [31]:
print(type(X))

<class 'torch.Tensor'>


In [32]:
#add tag and describtion
X = X.tag("Bob_iris-data")
y = y.tag("Bob_iris-target")

X = X.describe(
    f"This is a dataset for flower classification of {data_size} samples. 4 Features are sepal length (cm)," 
    "sepal width (cm), petal length (cm), petal width (cm)"
)
y = y.describe("Labels for flowers: Iris-setosa, Iris-versicolour, Iris-virginica")

In [33]:
#send pointable dataset to server, data scientist can access to the dataset only if (s)he make a request
data_pointer = X.send(duet, pointable=True)
target_pointer = y.send(duet, pointable=True)

In [34]:
# Once uploaded, the data owner can see the object stored in the tensor
duet.store

[<syft.proxy.torch.TensorPointer object at 0x7fe422d917c0>, <syft.proxy.torch.TensorPointer object at 0x7fe422d91af0>]

In [35]:
# To see it in a human-readable format, data owner can also pretty-print the tensor information
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 2585083ae8404693afcb7b6c9315bba3>,[Bob_iris-data],This is a dataset for flower classification of...,<class 'torch.Tensor'>
1,<UID: d5e32a2637b14e3c9e98d99bcb308a13>,[Bob_iris-target],"Labels for flowers: Iris-setosa, Iris-versicol...",<class 'torch.Tensor'>


## PART 3: Add Request Handlers

The Data Science Notebook makes a number of requests to access data. You can view these requests with:  
```
duet.requests.pandas
```
Then manually accept or deny them with:
```
duet.requests[0].accept()
```
However for training loops, this can be slow and tedious so the below code will create request handlers which will automatically respond with `accept` or `deny` depending on the name of the request.

In [36]:
duet.requests.pandas

""


In [37]:
# currently we do not have any protection from the handler, handler allows any requests
duet.requests.add_handler(
    action="accept",
    print_local=True,  # print the result in your notebook
)

[2022-07-10T15:39:50.027822+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 599cd2f5bcb9428c80f8d0ce9b542c13>: To evaluate training progress
Value: <Storable: 1.0737802982330322>
[2022-07-10T15:39:51.140543+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: c5b52f173eb94f6bb1295da7cb2b1fdc>: To evaluate training progress
Value: <Storable: 1.8612384796142578>
[2022-07-10T15:39:52.264122+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 099d33a5ead64606bfb03cfe7d9fcc3e>: To evaluate training progress
Value: <Storable: 2.5130488872528076>
[2022-07-10T15:39:53.374626+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 31a2ebd7a4544059b484a605a951c9d2>: To evaluate training progress
Value: <Storable: 0.7627922296524048>
[2022-07-10T15:39:54.491837+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 7d83b34fb9504bf2aecd0dc6698d6841>: To evaluate training progress
Value: <Storable: 0.7989017367362976>
[2022-07-10T15:39:55.615213+0300][CRITICAL][logger]][11

[2022-07-10T15:47:08.435359+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 21eaecc19d1c4104b8da2de97c4962d2>:
Value: <Storable: 'in_features=4, out_features=20, bias=True'>
[2022-07-10T15:47:09.547555+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 5e3d8bb910384922b574f654fe60acf4>:
Value: <Storable: OrderedDict([('weight', tensor([[ 0.1302, -0.4009, -0.7935, -0.1886],    [ 0.0487, -0.2635, -0.7329, -0.1551],    [-0.0434, 0.1223, 0.0178, -0.2859],    [-0.1380, -0.4091, 0.0014, 0.4134],    [-0.2216, -0.5422, -0.1289, -1.0081],    [-0.1855, 0.0480, -0.4581, -0.2016],    [ 0.2879, -0.3886, 0.1780, -0.3081],    [-0.2686, -0.3616, -0.3187, 0.0046],    [-0.4551, 0.3928, 0.3300, -0.1806],    [-0.2763, 0.0020, -0.5224, -0.8336],    [ 0.0921, -0.1488, -0.3658, -0.3189],    [ 0.0949, 0.0432, -0.4005, -0.0719],    [-0.2105, -0.2670, -0.2767, -0.1688],    [-0.0371, 0.0421, -0.3847, 0.3509],    [-0.1716, 0.1030, -0.4189, -0.8018],    [-0.4986, -0.6482, -0.0982, -0.4327],    [-0

[2022-07-10T15:47:11.868165+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: de2fd9c886a44597aadbb0f65933d968>:
Value: <Storable: 'in_features=30, out_features=30, bias=True'>
[2022-07-10T15:47:12.009457+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 1be5e471b43745af8b024eb79fd29880>:
Value: <Storable: OrderedDict([('weight', tensor([[-2.4456e-01, 1.4185e-01, -3.8364e-01, 1.5806e-01, -3.5345e-02,     -3.2282e-01, -3.0535e-01, -2.9788e-01, -3.9436e-01, -3.3533e-01,     -3.8407e-01, 9.7338e-02, -1.4347e-03, -1.7332e-01, 1.0270e-01,     -2.8640e-01, -3.6029e-01, -1.8037e-01, 1.5684e-02, -4.9432e-01,     -4.3744e-02, -1.2900e-01, -1.2699e-01, 4.0122e-02, -5.7254e-01,     4.6783e-01, 8.3722e-02, -4.7856e-01, -5.2502e-01, -2.7780e-01],    [-1.7194e-01, -3.3313e-01, -1.4870e-01, 4.3369e-02, 1.7588e-01,     -1.7255e-01, -3.5624e-01, -1.8169e-02, -3.8673e-01, 3.2997e-02,     4.3206e-02, -2.0521e-01, -1.5620e-01, -2.3461e-01, -1.3103e-01,     -1.8411e-01, -1.2944e-01, -1.5121

[2022-07-10T15:47:13.104403+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: e44ee7f7cc6a491fb6d48ec1e5ddab65>:
Value: <Storable: 'in_features=30, out_features=3, bias=True'>
[2022-07-10T15:47:13.238294+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 02287c6a61de416e8f366be79678aed3>:
Value: <Storable: OrderedDict([('weight', tensor([[-0.3073, -0.2195, -0.6621, -0.6757, -0.6457, -0.2257, -0.3895, -0.1424,     -0.5405, -0.1943, -0.4839, -0.5164, -0.3006, -0.0097, -0.5954, -0.2253,     -0.6804, -0.4563, -0.5497, -0.7225, -0.7032, -0.2408, -0.2170, -0.5163,     -0.3981, 0.0301, 0.0374, -0.6564, -0.6270, -0.0908],    [ 0.5855, -0.3229, 0.1705, 0.2246, 0.1863, -0.1655, 0.2479, 0.3309,     0.2397, 0.2066, 0.1336, 0.2384, 0.4320, -0.0977, 0.1699, -0.3788,     -0.2328, 0.3417, -0.2312, 0.1439, 0.0630, 0.2593, 0.1393, 0.1968,     0.2013, 0.4767, -0.0732, 0.0161, 0.0862, -0.0436],    [ 0.4648, 0.1462, 0.0646, 0.1235, 0.2031, 0.2827, 0.5436, -0.3386,     0.1112, -0.0824, 0.0584

[2022-07-10T15:47:21.740755+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 0209d1b9a557454b999fab4546c024fa>:
Value: <Storable: 'in_features=30, out_features=30, bias=True'>
[2022-07-10T15:47:22.873017+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 7a79e641615a46539eeacfbce9ca96ba>:
Value: <Storable: OrderedDict([('weight', tensor([[-2.4456e-01, 1.4185e-01, -3.8364e-01, 1.5806e-01, -3.5345e-02,     -3.2282e-01, -3.0535e-01, -2.9788e-01, -3.9436e-01, -3.3533e-01,     -3.8407e-01, 9.7338e-02, -1.4347e-03, -1.7332e-01, 1.0270e-01,     -2.8640e-01, -3.6029e-01, -1.8037e-01, 1.5684e-02, -4.9432e-01,     -4.3744e-02, -1.2900e-01, -1.2699e-01, 4.0122e-02, -5.7254e-01,     4.6783e-01, 8.3722e-02, -4.7856e-01, -5.2502e-01, -2.7780e-01],    [-1.7194e-01, -3.3313e-01, -1.4870e-01, 4.3369e-02, 1.7588e-01,     -1.7255e-01, -3.5624e-01, -1.8169e-02, -3.8673e-01, 3.2997e-02,     4.3206e-02, -2.0521e-01, -1.5620e-01, -2.3461e-01, -1.3103e-01,     -1.8411e-01, -1.2944e-01, -1.5121

...

[2022-07-10T16:01:51.936047+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: e85d9e7b373c435ba8cabbf5c5572ce4>:
Value: <Storable: 'in_features=4, out_features=20, bias=True'>
[2022-07-10T16:01:53.050006+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: ff10e3ddcc234431890d3d30af3c4d9d>:
Value: <Storable: OrderedDict([('weight', tensor([[ 0.1302, -0.4009, -0.7935, -0.1886],    [ 0.0487, -0.2635, -0.7329, -0.1551],    [-0.0434, 0.1223, 0.0178, -0.2859],    [-0.1380, -0.4091, 0.0014, 0.4134],    [-0.2216, -0.5422, -0.1289, -1.0081],    [-0.1855, 0.0480, -0.4581, -0.2016],    [ 0.2879, -0.3886, 0.1780, -0.3081],    [-0.2686, -0.3616, -0.3187, 0.0046],    [-0.4551, 0.3928, 0.3300, -0.1806],    [-0.2763, 0.0020, -0.5224, -0.8336],    [ 0.0921, -0.1488, -0.3658, -0.3189],    [ 0.0949, 0.0432, -0.4005, -0.0719],    [-0.2105, -0.2670, -0.2767, -0.1688],    [-0.0371, 0.0421, -0.3847, 0.3509],    [-0.1716, 0.1030, -0.4189, -0.8018],    [-0.4986, -0.6482, -0.0982, -0.4327],    [-0

[2022-07-10T16:01:56.363174+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: f3619aa10dd34b35b929e33f8fdbf543>:
Value: <Storable: 'in_features=30, out_features=30, bias=True'>
[2022-07-10T16:01:57.489043+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 2c2d063e0a7a4a4b84866078f6ba1d76>:
Value: <Storable: OrderedDict([('weight', tensor([[-2.4456e-01, 1.4185e-01, -3.8364e-01, 1.5806e-01, -3.5345e-02,     -3.2282e-01, -3.0535e-01, -2.9788e-01, -3.9436e-01, -3.3533e-01,     -3.8407e-01, 9.7338e-02, -1.4347e-03, -1.7332e-01, 1.0270e-01,     -2.8640e-01, -3.6029e-01, -1.8037e-01, 1.5684e-02, -4.9432e-01,     -4.3744e-02, -1.2900e-01, -1.2699e-01, 4.0122e-02, -5.7254e-01,     4.6783e-01, 8.3722e-02, -4.7856e-01, -5.2502e-01, -2.7780e-01],    [-1.7194e-01, -3.3313e-01, -1.4870e-01, 4.3369e-02, 1.7588e-01,     -1.7255e-01, -3.5624e-01, -1.8169e-02, -3.8673e-01, 3.2997e-02,     4.3206e-02, -2.0521e-01, -1.5620e-01, -2.3461e-01, -1.3103e-01,     -1.8411e-01, -1.2944e-01, -1.5121

[2022-07-10T16:01:58.577508+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 31fd848f3b0a4828bba7a7fe27a8b793>:
Value: <Storable: 'in_features=30, out_features=3, bias=True'>
[2022-07-10T16:01:59.694234+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 1789a1c7767446e3848f0c6f9c6e057a>:
Value: <Storable: OrderedDict([('weight', tensor([[-0.3073, -0.2195, -0.6621, -0.6757, -0.6457, -0.2257, -0.3895, -0.1424,     -0.5405, -0.1943, -0.4839, -0.5164, -0.3006, -0.0097, -0.5954, -0.2253,     -0.6804, -0.4563, -0.5497, -0.7225, -0.7032, -0.2408, -0.2170, -0.5163,     -0.3981, 0.0301, 0.0374, -0.6564, -0.6270, -0.0908],    [ 0.5855, -0.3229, 0.1705, 0.2246, 0.1863, -0.1655, 0.2479, 0.3309,     0.2397, 0.2066, 0.1336, 0.2384, 0.4320, -0.0977, 0.1699, -0.3788,     -0.2328, 0.3417, -0.2312, 0.1439, 0.0630, 0.2593, 0.1393, 0.1968,     0.2013, 0.4767, -0.0732, 0.0161, 0.0862, -0.0436],    [ 0.4648, 0.1462, 0.0646, 0.1235, 0.2031, 0.2827, 0.5436, -0.3386,     0.1112, -0.0824, 0.0584

[2022-07-10T16:02:05.233648+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: f029f7c674ca4a5e9a20b01a44b75fc2>:
Value: <Storable: 'in_features=30, out_features=30, bias=True'>
[2022-07-10T16:02:06.355486+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: fd9d7ad6e2c749b29d42c881011c1a63>:
Value: <Storable: OrderedDict([('weight', tensor([[-2.4456e-01, 1.4185e-01, -3.8364e-01, 1.5806e-01, -3.5345e-02,     -3.2282e-01, -3.0535e-01, -2.9788e-01, -3.9436e-01, -3.3533e-01,     -3.8407e-01, 9.7338e-02, -1.4347e-03, -1.7332e-01, 1.0270e-01,     -2.8640e-01, -3.6029e-01, -1.8037e-01, 1.5684e-02, -4.9432e-01,     -4.3744e-02, -1.2900e-01, -1.2699e-01, 4.0122e-02, -5.7254e-01,     4.6783e-01, 8.3722e-02, -4.7856e-01, -5.2502e-01, -2.7780e-01],    [-1.7194e-01, -3.3313e-01, -1.4870e-01, 4.3369e-02, 1.7588e-01,     -1.7255e-01, -3.5624e-01, -1.8169e-02, -3.8673e-01, 3.2997e-02,     4.3206e-02, -2.0521e-01, -1.5620e-01, -2.3461e-01, -1.3103e-01,     -1.8411e-01, -1.2944e-01, -1.5121

[2022-07-10T16:02:07.438294+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 5117e219b7ef472591ba798472013b1a>:
Value: <Storable: 'in_features=30, out_features=3, bias=True'>
[2022-07-10T16:02:08.561719+0300][CRITICAL][logger]][1151458] > HANDLER Request <UID: 9acafaa3ae6b40ca8eb74b06d3576c55>:
Value: <Storable: OrderedDict([('weight', tensor([[-0.3073, -0.2195, -0.6621, -0.6757, -0.6457, -0.2257, -0.3895, -0.1424,     -0.5405, -0.1943, -0.4839, -0.5164, -0.3006, -0.0097, -0.5954, -0.2253,     -0.6804, -0.4563, -0.5497, -0.7225, -0.7032, -0.2408, -0.2170, -0.5163,     -0.3981, 0.0301, 0.0374, -0.6564, -0.6270, -0.0908],    [ 0.5855, -0.3229, 0.1705, 0.2246, 0.1863, -0.1655, 0.2479, 0.3309,     0.2397, 0.2066, 0.1336, 0.2384, 0.4320, -0.0977, 0.1699, -0.3788,     -0.2328, 0.3417, -0.2312, 0.1439, 0.0630, 0.2593, 0.1393, 0.1968,     0.2013, 0.4767, -0.0732, 0.0161, 0.0862, -0.0436],    [ 0.4648, 0.1462, 0.0646, 0.1235, 0.2031, 0.2827, 0.5436, -0.3386,     0.1112, -0.0824, 0.0584